In [ ]:
import pandas as pd
import json
from tqdm import tqdm
import tensorflow as tf
import numpy as np

prefix = "/content/drive/MyDrive/nlp_project/"


In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')
# %cd '/content/drive/My Drive/path/to/my/project/folder'

# !pip install cache-magic
# import cache_magic

# !mkdir .cache
# !ln -s '/content/drive/My Drive/path/to/my/project/folder/.cache' /content/.cache

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
train_path = prefix + "train.json"
# test_path = prefix + "dev.json"

In [ ]:
# train = pd.read_json(train_path)
# test = pd.read_json(test_path)

In [ ]:
# train.head()

In [ ]:
with open(train_path) as data_file:    
    data = json.load(data_file)  

df = pd.json_normalize(data, record_path=[["annotations"]])
print (df.head())

                                              result
0  [{'id': 'd7a902fe9c23417499a7ef782f9fbdeb', 't...
1  [{'id': 'ac4523a0252e4007986cefbd6d5f571a', 'v...
2  [{'id': '43499bd62ea94624b2f38f4cbc677913', 'v...
3  [{'id': 'ec5e65782b1949e4a5445a2115ab5382', 'v...
4  [{'id': '7323f9247fbc4618bf006ef103d7cb3a', 't...


In [ ]:
for i, row in df.iterrows():
  print(row, row.dtype)
  break

result    [{'id': 'd7a902fe9c23417499a7ef782f9fbdeb', 't...
Name: 0, dtype: object object


In [ ]:
print(data[0]["annotations"][0]["result"][0]["value"]["text"])
print(data[0]["annotations"][0]["result"][0]["value"]["labels"][0])

      IN THE HIGH COURT OF KARNATAKA,
          CIRCUIT BENCH AT GULBARGA

DATED THIS THE 22ND DAY OF FEBRUARY, 2013
PREAMBLE


In [ ]:
x = pd.DataFrame(columns = ["text", "output"])

In [ ]:
# p = 0
# for i in tqdm(range(len(data))):
#   for j in range(len(data[i]["annotations"])):
#     for k in range(len(data[i]["annotations"][j]["result"])):
#       text = data[i]["annotations"][j]["result"][k]["value"]["text"]
#       output = data[i]["annotations"][j]["result"][k]["value"]["labels"][0]
#       x.loc[p] = [text, output]
#       p+=1

import pickle

# obj0, obj1, obj2 are created here...

# Saving the objects:
# with open(prefix + 'train_data.pkl', 'wb') as f:  # Python 3: open(..., 'wb')
#     pickle.dump(x, f)

# Getting back the objects:
with open(prefix + 'train_data.pkl', 'rb') as f:  # Python 3: open(..., 'rb')
    x = pickle.load(f)



In [ ]:
x.head()

,text,output,label
0,"IN THE HIGH COURT OF KARNATAKA,\n ...",PREAMBLE,1
1,\n\n BEFORE\n\nTHE HON'BLE MR.JUSTICE ANA...,PREAMBLE,1
2,This Criminal Appeal is filed under Section 37...,PREAMBLE,1
3,\n\n This appeal coming on for hearing t...,PREAMBLE,1
4,\n Heard the learned Counsel for the app...,NONE,11


In [ ]:
t = tf.keras.preprocessing.text.Tokenizer()
t.fit_on_texts(x["text"].tolist())
vocab_size = len(t.word_index) + 1
# integer encode the documents
encoded_docs = t.texts_to_sequences(x["text"].tolist())
# print(encoded_docs)

In [ ]:
max_len = 0
for i in encoded_docs:
  max_len = max(max_len, len(i))
  # max_len = 380

In [ ]:
max_len = 380

In [ ]:
padded_docs = tf.keras.utils.pad_sequences(encoded_docs, maxlen=max_len, padding='post')

In [ ]:
label = list(set(x["output"].tolist()))

In [ ]:
d = {}
for i in range(len(label)):
  d[label[i]] = i

In [ ]:
x["label"] = x["output"].map(d)

In [ ]:
# train_save_path = "/content/drive/MyDrive/nlp_project/train.csv"
# x.to_csv(train_save_path, index=False)

In [ ]:
final_label = x["label"].tolist()

In [ ]:
# final_label

In [ ]:
# we have (padded_docs, final_label)

In [ ]:
# !wget http://nlp.stanford.edu/data/glove.6B.zip
# !unzip -q glove.6B.zip

In [ ]:
# train_save_path = "/content/drive/MyDrive/nlp_project/train.csv"
# train_data = pd.read_csv(train_save_path)

In [ ]:
# train_data.head()
# final_labels = train_data["label"].tolist()

In [ ]:
glove_embedding_path = "/content/drive/MyDrive/nlp_project/glove.6B.100d.txt"

In [ ]:
embeddings_index = dict()
f = open(glove_embedding_path)
for line in f:
	values = line.split()
	word = values[0]
	coefs = np.asarray(values[1:], dtype='float32')
	embeddings_index[word] = coefs
f.close()

In [ ]:
embedding_matrix = np.zeros((vocab_size, 100))
for word, i in t.word_index.items():
	embedding_vector = embeddings_index.get(word)
	if embedding_vector is not None:
		embedding_matrix[i] = embedding_vector

In [ ]:
final_labels = np.array(final_label)

In [ ]:
num_classes = len(d)

In [ ]:
final_labels = tf.keras.utils.to_categorical(final_labels, num_classes=num_classes)

x_test, x_train = padded_docs[:3000], padded_docs[3000:]
y_test, y_train = final_labels[:3000], final_labels[3000:]

In [ ]:
model = tf.keras.Sequential()
e = tf.keras.layers.Embedding(vocab_size, 100, weights=[embedding_matrix], input_length=max_len, trainable=False)
model.add(e)
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(num_classes, activation='softmax'))

# compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
# summarize the model
print(model.summary())

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, 380, 100)          2192000   
                                                                 
 flatten_4 (Flatten)         (None, 38000)             0         
                                                                 
 dense_4 (Dense)             (None, 13)                494013    
                                                                 
Total params: 2,686,013
Trainable params: 494,013
Non-trainable params: 2,192,000
_________________________________________________________________
None


In [ ]:
model.fit(x_train, y_train, epochs=10, verbose=2)
# evaluate the model

Epoch 1/10
813/813 - 7s - loss: 1.7072 - accuracy: 0.4881 - 7s/epoch - 9ms/step
Epoch 2/10
813/813 - 7s - loss: 1.2528 - accuracy: 0.5955 - 7s/epoch - 8ms/step
Epoch 3/10
813/813 - 14s - loss: 1.1062 - accuracy: 0.6330 - 14s/epoch - 17ms/step
Epoch 4/10
813/813 - 7s - loss: 1.0053 - accuracy: 0.6654 - 7s/epoch - 8ms/step
Epoch 5/10
813/813 - 6s - loss: 0.9408 - accuracy: 0.6842 - 6s/epoch - 8ms/step
Epoch 6/10
813/813 - 7s - loss: 0.8897 - accuracy: 0.6968 - 7s/epoch - 8ms/step
Epoch 7/10
813/813 - 7s - loss: 0.8466 - accuracy: 0.7173 - 7s/epoch - 8ms/step
Epoch 8/10
813/813 - 7s - loss: 0.8104 - accuracy: 0.7246 - 7s/epoch - 8ms/step
Epoch 9/10
813/813 - 7s - loss: 0.7847 - accuracy: 0.7329 - 7s/epoch - 8ms/step
Epoch 10/10
813/813 - 7s - loss: 0.7572 - accuracy: 0.7423 - 7s/epoch - 8ms/step


In [ ]:
y_pred = model.predict(x_test, verbose=2)

94/94 - 1s - 571ms/epoch - 6ms/step


In [ ]:
Y_pred = np.argmax(y_pred,axis=1)
Y_test = np.argmax(y_test,axis=1)

In [ ]:
from sklearn.metrics import f1_score
score=f1_score(Y_test, Y_pred, average='macro')

In [ ]:
score

0.2891595644308975

In [ ]:
model = tf.keras.Sequential()
e = tf.keras.layers.Embedding(vocab_size, 100, weights=[embedding_matrix], input_length=max_len, trainable=False)
model.add(e)
# model.add(tf.keras.layers.Flatten())

model.add(tf.keras.layers.Conv1D(32, 3, activation='relu'))

# forward_layer = tf.keras.layers.LSTM(1, return_sequences=True)
# backward_layer = tf.keras.layers.LSTM(2, activation='relu', return_sequences=True,
#                       go_backwards=True)
# model.add(tf.keras.layers.Bidirectional(forward_layer, backward_layer=backward_layer,
#                         input_shape=(1, 1)))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(num_classes, activation='softmax'))

# compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
# summarize the model
print(model.summary())

Model: "sequential_16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_16 (Embedding)    (None, 380, 100)          2192000   
                                                                 
 conv1d_10 (Conv1D)          (None, 378, 32)           9632      
                                                                 
 flatten_14 (Flatten)        (None, 12096)             0         
                                                                 
 dense_14 (Dense)            (None, 13)                157261    
                                                                 
Total params: 2,358,893
Trainable params: 166,893
Non-trainable params: 2,192,000
_________________________________________________________________
None


In [ ]:
model.fit(x_train, y_train, epochs=10, verbose=2)


Epoch 1/10
813/813 - 3s - loss: 1.6416 - accuracy: 0.4836 - 3s/epoch - 3ms/step
Epoch 2/10
813/813 - 2s - loss: 1.3365 - accuracy: 0.5675 - 2s/epoch - 3ms/step
Epoch 3/10
813/813 - 2s - loss: 1.1786 - accuracy: 0.6161 - 2s/epoch - 3ms/step
Epoch 4/10
813/813 - 2s - loss: 1.0764 - accuracy: 0.6485 - 2s/epoch - 3ms/step
Epoch 5/10
813/813 - 2s - loss: 0.9964 - accuracy: 0.6686 - 2s/epoch - 3ms/step
Epoch 6/10
813/813 - 2s - loss: 0.9400 - accuracy: 0.6871 - 2s/epoch - 3ms/step
Epoch 7/10
813/813 - 2s - loss: 0.8910 - accuracy: 0.7030 - 2s/epoch - 3ms/step
Epoch 8/10
813/813 - 2s - loss: 0.8466 - accuracy: 0.7156 - 2s/epoch - 3ms/step
Epoch 9/10
813/813 - 2s - loss: 0.8112 - accuracy: 0.7260 - 2s/epoch - 3ms/step
Epoch 10/10
813/813 - 2s - loss: 0.7804 - accuracy: 0.7374 - 2s/epoch - 3ms/step


In [ ]:
len(x_train)

25986

In [ ]:
y_pred = model.predict(x_test, verbose=2)

94/94 - 0s - 206ms/epoch - 2ms/step


In [ ]:
Y_pred = np.argmax(y_pred,axis=1)
Y_test = np.argmax(y_test,axis=1)

In [ ]:
from sklearn.metrics import f1_score
score=f1_score(Y_test, Y_pred, average='macro')

In [ ]:
score

0.31580827210263746